<a href="https://colab.research.google.com/github/YasmeanMohammadi/algorithms/blob/main/StableMatching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import time

def gale_shapley(men_prefs, women_prefs):
    """
    Implements the Gale-Shapley stable matching algorithm.

    men_prefs: A list of preference lists for each man.
    women_prefs: A list of preference lists for each woman.

    Returns a stable matching as a vector.
    """
    n = len(men_prefs)
    free_men = list(range(n))  # All men start as free
    proposals = np.zeros((n, n), dtype=int)  # Track proposals (0 = not proposed, 1 = proposed)
    engaged = [-1] * n  # Stores woman's current match (-1 means single)
    women_ranks = np.argsort(women_prefs, axis=1)  # Fast lookup for ranking

    while free_men:
        man = free_men.pop(0)  # Take the first free man

        for woman in men_prefs[man]:
            if proposals[man][woman] == 0:  # If not yet proposed
                proposals[man][woman] = 1  # Mark as proposed
                current_partner = engaged[woman]

                if current_partner == -1:  # Woman is free
                    engaged[woman] = man
                    break

                # Check if she prefers the new proposer
                elif women_ranks[woman][man] < women_ranks[woman][current_partner]:
                    engaged[woman] = man  # She switches partners
                    free_men.append(current_partner)  # The rejected man is now free
                    break

    return np.array(engaged)  # Convert result into a vector

# Example with random preferences
np.random.seed(42)
n = 5  # Number of men and women
men_prefs = np.argsort(np.random.rand(n, n), axis=1)
women_prefs = np.argsort(np.random.rand(n, n), axis=1)

# Run Gale-Shapley algorithm and measure time
start_time = time.time()
stable_matching = gale_shapley(men_prefs, women_prefs)
end_time = time.time()

# Visualizing the proposal process
plt.figure(figsize=(6, 6))
plt.imshow(men_prefs, cmap="Blues", aspect="auto")
plt.colorbar(label="Rank in Preference List")
plt.xlabel("Women")
plt.ylabel("Men")
plt.title("Men's Preference Matrix")
plt.show()

plt.figure(figsize=(6, 6))
plt.imshow(women_prefs, cmap="Oranges", aspect="auto")
plt.colorbar(label="Rank in Preference List")
plt.xlabel("Men")
plt.ylabel("Women")
plt.title("Women's Preference Matrix")
plt.show()

# Visualizing the stable matching as a bipartite graph
G = nx.Graph()
men_nodes = [f"M{i}" for i in range(n)]
women_nodes = [f"W{i}" for i in range(n)]
G.add_nodes_from(men_nodes, bipartite=0)
G.add_nodes_from(women_nodes, bipartite=1)
G.add_edges_from([(f"M{stable_matching[i]}", f"W{i}") for i in range(n)])

plt.figure(figsize=(8, 6))
pos = {**{men_nodes[i]: (0, i) for i in range(n)}, **{women_nodes[i]: (1, i) for i in range(n)}}
nx.draw(G, pos, with_labels=True, node_size=2000, node_color='lightblue', edge_color='black', font_size=10)
plt.title("Stable Matching Bipartite Graph")
plt.show()

# Display results
print("Stable Matching (as vector):", stable_matching)
print("Execution Time:", end_time - start_time, "seconds")
